In [1]:
import pandas as pd
a=pd.read_table("/home/jash/Desktop/petplan/Profitability1.txt", sep='|', lineterminator='\n', error_bad_lines=False)

Skipping line 267909: expected 33 fields, saw 34

Skipping line 299539: expected 33 fields, saw 34

/home/jash/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
data=pd.DataFrame(data=a,index=None)
data = data.sample(frac=1).reset_index(drop=True)
import math
import numpy as np
data['LossRatio'].fillna(0, inplace=True)
data['L']=0
data.loc[data['LossRatio'] >8.451531 ,'L'] = 1
data.to_csv("/home/jash/Desktop/petplan/Profitability.csv")
Y=data['L']
y=pd.DataFrame(Y)
x=pd.DataFrame(data)

m=x.loc[x['PolicyForm']!='Introductory']
q=pd.DataFrame(m)
m2=q.loc[q['PolicyForm']!='Intro']
m3=pd.DataFrame(m2)
m4=m3.loc[m3['Quadrant']=='LowTenure-HighLR']
x=pd.DataFrame(m4)

print x.head()


     LossRatio  Duration                                     PetId  \
106   0.666673       279                           US2120882Winnie   
161  13.436798       301                            US2103207Lacey   
220  12.565342       155                          US1385556Belmont   
226   5.763426       549                               US1276202TJ   
239   3.236195       400  USB3F7E981869B4D44B064ADEDE8FC903AMillie   

      StartDate     EndDate                     LastPolicyRef  \
106  2017-10-22  2018-11-13                            551454   
161  2017-09-30  2018-09-29                            527677   
220  2015-08-10  2016-01-11                            129845   
226  2015-04-30  2016-10-29                            217998   
239  2015-02-23  2016-03-28  1A91F844BD2C40139710D3997278508C   

     TotalClaimsAmtPaid  TotalEarnedPremium                       BreedName  \
106             370.392          555.583123                 English Bulldog   
161            5443.200       

In [3]:
x['LossRatio'].max()

526.2746201058766

In [4]:
len(x.loc[x['L']==1])

4189

In [5]:
len(x)

16161

In [6]:

x.isnull().sum()

LossRatio                           0
Duration                            0
PetId                               0
StartDate                           0
EndDate                             0
LastPolicyRef                       0
TotalClaimsAmtPaid                  0
TotalEarnedPremium                  0
BreedName                           0
Currentage                          0
CopayPct                            0
Deductible                          0
PetType                             0
PolicyForm                          0
CarrierCd                           0
ClaimNumber                         0
ClaimAmount                         0
ClaimDetails                        0
ConditionGrp                        0
Claimcodecategory                   0
Severity                           46
CustomerNumber                     61
Country                            61
PhoneNumber                        92
Surname                            63
GivenName                          67
CustomerMail

In [7]:
x.drop(['LossRatio'],axis=1,inplace=True)
x.drop(['CustomerNumber','PhoneNumber','Surname','GivenName','CustomerMailingAddr_Addr1','CustomerMailingAddr_City','CustomerMailingAddr_StateProvCd',
       'CustomerMailingAddr_PostalCode','PetId','StartDate','EndDate','LastPolicyRef'],axis=1,inplace=True)



In [8]:
p=pd.DataFrame(x)

In [9]:
p.drop(['ClaimNumber','ClaimAmount','Severity','ClaimDetails','ConditionGrp','Claimcodecategory','claimdurationInception'],axis=1,inplace=True)

In [10]:
p.isnull().sum()

Duration               0
TotalClaimsAmtPaid     0
TotalEarnedPremium     0
BreedName              0
Currentage             0
CopayPct               0
Deductible             0
PetType                0
PolicyForm             0
CarrierCd              0
Country               61
Quadrant               0
churn\r                0
L                      0
dtype: int64

In [11]:


p['TotalClaimsAmtPaid'].fillna(0, inplace=True)

In [12]:
p.isnull().sum()

Duration               0
TotalClaimsAmtPaid     0
TotalEarnedPremium     0
BreedName              0
Currentage             0
CopayPct               0
Deductible             0
PetType                0
PolicyForm             0
CarrierCd              0
Country               61
Quadrant               0
churn\r                0
L                      0
dtype: int64

In [13]:
p.dropna(how='any',inplace=True)

In [14]:
p.isnull().sum()

Duration              0
TotalClaimsAmtPaid    0
TotalEarnedPremium    0
BreedName             0
Currentage            0
CopayPct              0
Deductible            0
PetType               0
PolicyForm            0
CarrierCd             0
Country               0
Quadrant              0
churn\r               0
L                     0
dtype: int64

In [15]:
p.drop(['Quadrant','TotalClaimsAmtPaid','TotalEarnedPremium'],axis=1,inplace=True)

In [16]:
print p.shape

(16100, 11)


In [17]:

import h2o
from h2o.estimators import H2ORandomForestEstimator
h2o.init()
df = h2o.H2OFrame(p)
y = 'L'
x=df.col_names
print df
x.remove(y)
df[y] = df[y].asfactor()
train, valid, test = df.split_frame(ratios=[.8,.1])
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
gbm_cv3 = H2OGradientBoostingEstimator(nfolds=3,max_depth=4,ntrees=130)
gbm_cv3.train(x=x, y=y, training_frame=train)
## Getting all cross validated models 
all_models = gbm_cv3.cross_validation_models()
print("Total cross validation models: " + str(len(all_models)))

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,21 hours 50 mins
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,4 months and 28 days !!!
H2O cluster name:,H2O_from_python_jash_sdf687
H2O cluster total nodes:,1
H2O cluster free memory:,0
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


/home/jash/anaconda3/envs/py27/lib/python2.7/site-packages/h2o/h2o.py:562: UserWarning: [Lwater.parser.ParseWriter$ParseErr;@150e033a
  warnings.warn(w)


H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Total input file size of 1.1 MB is much larger than total cluster memory of Zero  , please use either a larger cluster or smaller data.
  Request: POST /3/Parse
    data: {u'source_frames': u'["upload_8b2b4eaf8f4bb83ea711548954de4a77"]', u'column_names': u'["Duration","BreedName","Currentage","CopayPct","Deductible","PetType","PolicyForm","CarrierCd","Country","churn\r","L"]', u'check_header': '1', u'destination_frame': 'Key_Frame__upload_8b2b4eaf8f4bb83ea711548954de4a77.hex', u'single_quotes': 'False', u'parse_type': 'CSV', u'separator': '44', u'number_columns': '11', u'chunk_size': '4194304', u'column_types': u'["Numeric","Enum","Numeric","Numeric","Numeric","Enum","Numeric","Enum","Enum","Numeric","Numeric"]', u'delete_on_done': 'True', u'blocking': 'False'}


In [ ]:
## Depth 10 is usually plenty of depth for most datasets, but you never know
hyper_params = {'max_depth' : [1,2,3,4,5,6,7,8,9,10]}
#hyper_params = {max_depth = [4,6,8,12,16,20]} ##faster for larger datasets

#Build initial GBM Model
gbm_grid = H2OGradientBoostingEstimator(
        ## more trees is better if the learning rate is small enough 
        ## here, use "more than enough" trees - we have early stopping
        ntrees=10000,
        ## smaller learning rate is better
        ## since we have learning_rate_annealing, we can afford to start with a 
        #bigger learning rate
        learn_rate=0.05,
        ## learning rate annealing: learning_rate shrinks by 1% after every tree 
        ## (use 1.00 to disable, but then lower the learning_rate)
        learn_rate_annealing = 0.99,
        ## sample 80% of rows per tree
        sample_rate = 0.8,
        ## sample 80% of columns per split
        col_sample_rate = 0.8,
        ## fix a random number generator seed for reproducibility
        seed = 1234,
        ## score every 10 trees to make early stopping reproducible 
        #(it depends on the scoring interval)
        score_tree_interval = 10, 
        ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
        #5 consecutive scoring events
        stopping_rounds = 5,
        stopping_metric = "AUC",
        stopping_tolerance = 1e-4)

#Build grid search with previously made GBM and hyper parameters
grid = H2OGridSearch(gbm_grid,hyper_params,
                         grid_id = 'depth_grid',
                         search_criteria = {'strategy': "Cartesian"})


#Train grid search
grid.train(x=x, 
           y=y,
           training_frame = train,
           validation_frame = valid)

In [ ]:
## by default, display the grid search results sorted by increasing logloss (since this is a classification task)
print grid

In [ ]:

## sort the grid models by decreasing AUC
sorted_grid = grid.get_grid(sort_by='auc',decreasing=True)
print(sorted_grid)

In [ ]:

max_depths = sorted_grid.sorted_metric_table()['max_depth'][0:5]
new_max = int(max(max_depths, key=int))
new_min = int(min(max_depths, key=int))

print "MaxDepth", new_max
print "MinDepth", new_min

In [ ]:
new_min=1
new_max=5
# create hyperameter and search criteria lists (ranges are inclusive..exclusive))
hyper_params_tune = {'max_depth' : list(range(new_min,new_max+1,1)),
                'sample_rate': [x/100. for x in range(20,101)],
                'col_sample_rate' : [x/100. for x in range(20,101)],
                'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(train.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,11)],
                'nbins_cats': [2**x for x in range(4,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}
search_criteria_tune = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 3600,  ## limit the runtime to 60 minutes
                   'max_models': 100,  ## build no more than 100 models
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "AUC",
                   'stopping_tolerance': 1e-3 }
       
           

In [ ]:
gbm_final_grid = H2OGradientBoostingEstimator(distribution='bernoulli',
                    ## more trees is better if the learning rate is small enough 
                    ## here, use "more than enough" trees - we have early stopping
                    ntrees=10000,
                    ## smaller learning rate is better
                    ## since we have learning_rate_annealing, we can afford to start with a 
                    #bigger learning rate
                    learn_rate=0.05,
                    ## learning rate annealing: learning_rate shrinks by 1% after every tree 
                    ## (use 1.00 to disable, but then lower the learning_rate)
                    learn_rate_annealing = 0.99,
                    ## score every 10 trees to make early stopping reproducible 
                    #(it depends on the scoring interval)
                    score_tree_interval = 10,
                    ## fix a random number generator seed for reproducibility
                    seed = 1234,
                    ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
                    #5 consecutive scoring events
                    stopping_rounds = 5,
                    stopping_metric = "AUC",
                    stopping_tolerance = 1e-4)
            
#Build grid search with previously made GBM and hyper parameters
final_grid = H2OGridSearch(gbm_final_grid, hyper_params = hyper_params_tune,
                                    grid_id = 'final_grid',
                           search_criteria = search_criteria_tune)
#Train grid search


x=df.col_names
x.remove(y)
df[y] = df[y].asfactor()
train, valid, test = df.split_frame(ratios=[.8,.1])


final_grid.train(x=x, 
           y=y,
           training_frame = train,
           validation_frame = valid)

print final_grid

In [ ]:

## Sort the grid models by AUC
sorted_final_grid = final_grid.get_grid(sort_by='auc',decreasing=True)

print sorted_final_grid

In [ ]:

#Get the best model from the list (the model name listed at the top of the table)
best_model = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][0])
performance_best_model = best_model.model_performance(test)
print performance_best_model.auc()

In [ ]:

params_list = []
for key, value in best_model.params.iteritems():
    params_list.append(str(key)+" = "+str(value['actual']))
params_list

In [ ]:
gbm = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][0])
#get the parameters from the Random grid search model and modify them slightly
params = gbm.params
new_params = {"nfolds":5, "model_id":None}
for key in new_params.keys():
    params[key]['actual'] = new_params[key] 
gbm_best = H2OGradientBoostingEstimator()


In [ ]:
for key in params.keys():
    if key in dir(gbm_best) and getattr(gbm_best,key) != params[key]['actual']:
        setattr(gbm_best,key,params[key]['actual'])

In [ ]:
gbm_best.train(x=x, y=y, training_frame=df)

In [ ]:
gbm_best.confusion_matrix()
performance = gbm_best.model_performance(test_data=test)
print performance


In [ ]:
import matplotlib.pyplot as plt
gbm_best.varimp_plot()
gbm_best.varimp()

In [ ]:
import subprocess
from IPython.display import Image

In [ ]:
mojo_file_name = "/home/jash/Desktop/petplan/treeshyper/my_gbm_mojo.zip"
h2o_jar_path= '/home/jash/Desktop/petplan/trees/h2o-3.20.0.4/h2o.jar'
mojo_full_path = mojo_file_name
gv_file_path = "/home/jash/Desktop/petplan/treeshyper/my_gbm_graph.gv"

In [ ]:
image_file_name = "/home/jash/Desktop/petplan/treeshyper/my_gbm_tree_best"

In [ ]:
gbm_cv3.download_mojo(mojo_file_name)

In [ ]:

def generateTree(h2o_jar_path, mojo_full_path, gv_file_path, image_file_path, tree_id = 0):
    image_file_path = image_file_path + "_" + str(tree_id) + ".png"
    result = subprocess.call(["java", "-cp", h2o_jar_path, "hex.genmodel.tools.PrintMojo", "--tree", str(tree_id), "-i", mojo_full_path , "-o", gv_file_path ], shell=False)
    result = subprocess.call(["ls",gv_file_path], shell = False)
    if result is 0:
        print("Success: Graphviz file " + gv_file_path + " is generated.")
    else: 
        print("Error: Graphviz file " + gv_file_path + " could not be generated.")

In [ ]:
def generateTreeImage(gv_file_path, image_file_path, tree_id):
    image_file_path = image_file_path + "_" + str(tree_id) + ".png"
    result = subprocess.call(["dot", "-Tpng", gv_file_path, "-o", image_file_path], shell=False)
    result = subprocess.call(["ls",image_file_path], shell = False)
    if result is 0:
        print("Success: Image File " + image_file_path + " is generated.")
        print("Now you can execute the follow line as-it-is to see the tree graph:") 
        print("Image(filename='" + image_file_path + "\')")
    else:
        print("Error: Image file " + image_file_path + " could not be generated.")

In [ ]:

generateTree(h2o_jar_path, mojo_full_path, gv_file_path, image_file_name, 0)

In [ ]:
generateTreeImage(gv_file_path, image_file_name, 0)

In [ ]:
Image(filename='/home/jash/Desktop/petplan/treeshyper/my_gbm_tree_best.png')